In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data=Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\집/.surprise_data/ml-100k


In [3]:
trainset,testset=train_test_split(data,test_size=.25,random_state=0)

In [4]:
algo=SVD()
algo.fit(trainset)

In [9]:
predictions=algo.test(testset)
print('prediction type :',type(predictions),'size:',len(predictions))
print('Precede 5 \n',predictions[:5])
# Prediction object - Surprise's python list
# uid == userid , iid == itemid, r_ui == rating, est == estimated rating, detail {was_im--} == if true can't predict

prediction type : <class 'list'> size: 25000
Precede 5 
 [Prediction(uid='120', iid='282', r_ui=4.0, est=3.527450187105002, details={'was_impossible': False}), Prediction(uid='882', iid='291', r_ui=4.0, est=3.7092038003292718, details={'was_impossible': False}), Prediction(uid='535', iid='507', r_ui=5.0, est=4.343963702179069, details={'was_impossible': False}), Prediction(uid='697', iid='244', r_ui=5.0, est=3.2760702293290906, details={'was_impossible': False}), Prediction(uid='751', iid='385', r_ui=4.0, est=3.289423544742187, details={'was_impossible': False})]


In [10]:
[(pred.uid,pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.527450187105002),
 ('882', '291', 3.7092038003292718),
 ('535', '507', 4.343963702179069)]

In [11]:
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [12]:
accuracy.rmse(predictions)

RMSE: 0.9475


0.9475012906373043

In [14]:
import pandas as pd
ratings= pd.read_csv('../../data/ml-latest-small/ml-latest-small/ratings.csv')


In [17]:
ratings.to_csv('../../data/ml-latest-small/ml-latest-small/ratings_noh.csv',index=False,header=False)

In [18]:
from surprise import Reader

reader=Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('../../data/ml-latest-small/ml-latest-small/ratings_noh.csv',reader=reader)

In [20]:
trainset,testset = train_test_split(data, test_size=.25,random_state=0)

algo=SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [22]:
ratings=pd.read_csv('../../data/ml-latest-small/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5))

data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset=train_test_split(data,test_size=0.25,random_state=0)

algo=SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [24]:
from surprise.model_selection import cross_validate

ratings=pd.read_csv('../../data/ml-latest-small/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo=SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8725  0.8793  0.8707  0.8731  0.8714  0.8734  0.0031  
MAE (testset)     0.6688  0.6765  0.6677  0.6695  0.6730  0.6711  0.0032  
Fit time          6.58    6.59    6.58    6.54    6.63    6.58    0.03    
Test time         0.25    0.24    0.16    0.22    0.27    0.23    0.04    


{'test_rmse': array([0.87250088, 0.87933808, 0.8706915 , 0.87314737, 0.87138623]),
 'test_mae': array([0.66882694, 0.67646047, 0.66773788, 0.66951973, 0.67299742]),
 'fit_time': (6.582957744598389,
  6.59051775932312,
  6.5755534172058105,
  6.542913913726807,
  6.625675916671753),
 'test_time': (0.2459099292755127,
  0.23898100852966309,
  0.16292667388916016,
  0.21892023086547852,
  0.26643943786621094)}

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20,40,60],'n_factors':[50,100,200]}
gs=GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)


In [29]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8771237018114008
{'n_epochs': 20, 'n_factors': 50}


In [30]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='../../data/ml-latest-small/ml-latest-small/ratings_noh.csv',reader=reader)

trainset=data_folds.build_full_trainset()

In [31]:
algo=SVD(n_epochs=20,n_factors=50,random_state=0)
algo.fit(trainset)

In [32]:
movies=pd.read_csv('../../data/ml-latest-small/ml-latest-small/movies.csv')
movieIds = ratings[ratings['userId']==9]['movieId']

In [34]:
if movieIds[movieIds==42].count() == 0:
    print('No Rate')
print(movies[movies['movieId']==42])

No Rate
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [35]:
uid=str(9)
iid=str(42)
pred=algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [36]:
def get_unseen_surprise(ratings,movies,userId):
    seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies=movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('Number of Rating Movies :',len(seen_movies),'Number of Recommendation Movie :',len(unseen_movies),
         'Number of Total Movies :',len(total_movies))
    return unseen_movies
unseen_movies=get_unseen_surprise(ratings,movies,9)

Number of Rating Movies : 46 Number of Recommendation Movie : 9696 Number of Total Movies : 9742


In [41]:
import numpy as np

In [79]:
def recomm_movie_by_surprise(algo,userId,unseen_movies,top_n=10):
    predictions = [algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]
    #[prediction(uid iid est),prediction(uid iid est),,,,,,]
    # sort by est 
    
    def sorted_est(pred): #sort() function's attribute
        return pred.est
    
    predictions.sort(key=sorted_est,reverse=True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids=[int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id,title,rating) for id, title, rating in zip(top_movie_ids,top_movie_titles,top_movie_rating)]
    return top_movie_preds

unseen_movies=get_unseen_surprise(ratings,movies,9)
top_movies_preds=recomm_movie_by_surprise(algo,9,unseen_movies,top_n = 10)
longest=len(top_movies_preds[0][1])
for i in range(1,10):
    longest=longest if longest>len(top_movies_preds[i][1])\
    else (len(top_movies_preds[i][1]))
    
print('                ########## TOP - 10 Movies For You ##########')
for top_movie in top_movies_preds:
    print('{:{}}:{}'.format(top_movie[1],longest+1,np.round(top_movie[2],1)))

Number of Rating Movies : 46 Number of Recommendation Movie : 9696 Number of Total Movies : 9742
                ########## TOP - 10 Movies For You ##########
Usual Suspects, The (1995)                            :4.3
Star Wars: Episode IV - A New Hope (1977)             :4.3
Pulp Fiction (1994)                                   :4.3
Silence of the Lambs, The (1991)                      :4.2
Godfather, The (1972)                                 :4.2
Streetcar Named Desire, A (1951)                      :4.2
Star Wars: Episode V - The Empire Strikes Back (1980) :4.1
Star Wars: Episode VI - Return of the Jedi (1983)     :4.1
Goodfellas (1990)                                     :4.1
Glory (1989)                                          :4.1
